## Como gerar um diretório contendo os arquivos Fits


In [ ]:
# para instalar o astropy
# !pip install astropy
from astropy.io import fits
hdul = fits.open('specObj-dr17.fits')
hdul.info()

# ver as colunas 
print(hdul[1].columns)




In [ ]:
import os
print(os.getcwd())
# aqui você verá o diretório atual do seu pc, onde o arquivo .fits foi baixado

In [ ]:
import numpy as np
from astropy.io import fits
import threading
import requests
from collections import Counter
# Bibliotecas necessárias para o funcionamento do código

Agora vamos implementar um gerador de amostras 

In [ ]:

def gerador_de_amostras(sample_size, specobj_filename):
    catalog_file = specobj_filename
    try:
        hdul = fits.open(catalog_file)
        data = hdul[1].data  # Acessa a tabela do catálogo
        hdul.close()
        
        total_objects = len(data)
        sample_size = sample_size
        if total_objects < sample_size:
            print(f"O catálogo contém apenas {total_objects} objetos, não é possível extrair uma amostra de {sample_size}.")
        else:
            
            indices = np.random.choice(total_objects, sample_size, replace=False)
            random_sample = data[indices]
            print(f"Amostra aleatória de {sample_size} objetos extraída com sucesso.")
            # Agora você pode utilizar random_sample para suas análises
    except Exception as e:
        print(f"Erro: {e}")
    if 'CLASS' in random_sample.names:
        classes = random_sample['CLASS']
        counts = Counter(classes)
        print("Número de objetos por classe na amostra:")
        for classe, num in counts.items():
            print(f"{classe}: {num}")
    else:
        print("A coluna 'CLASS' não foi encontrada na amostra.")
    return random_sample

In [ ]:
def baixar_espectro(target_plate, target_mjd, target_fiberid, classe, subclass,redshift, download_dir):
    plate_str = f'{target_plate:04d}'
    mjd_str = f'{target_mjd}'
    fiberid_str = f'{target_fiberid:04d}'
    classe_str = classe
    subclass_str = subclass
    redshift_str = f'{redshift:.5f}'
    spectrum_filename = f'spec-{plate_str}-{mjd_str}-{fiberid_str}-{classe_str}-{subclass_str}-{redshift_str}.fits'
    full_spectrum_path = os.path.join(download_dir, spectrum_filename)
    sdss_sas_url_base = 'https://dr17.sdss.org/optical/spectrum/view/data/format=fits/spec=lite'
    download_url = (
        f'{sdss_sas_url_base}?'
        f'plateid={plate_str}'
        f'&mjd={mjd_str}'
        f'&fiberid={fiberid_str}'
    )

    print(f"Thread {threading.current_thread().name}: Tentando baixar {spectrum_filename} de {download_url}")
    try:
        response = requests.get(download_url, stream=True)
        response.raise_for_status()  # Lança uma exceção para códigos de status ruins (4xx ou 5xx)
        with open(full_spectrum_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Thread {threading.current_thread().name}: Download de {spectrum_filename} concluído.")
    except requests.exceptions.RequestException as e:
        print(f"Thread {threading.current_thread().name}: Erro ao baixar {spectrum_filename}: {e}")
        if os.path.exists(full_spectrum_path):
            try:
                os.remove(full_spectrum_path)
            except OSError as e:
                print(f"Thread {threading.current_thread().name}: Erro ao remover arquivo incompleto {full_spectrum_path}: {e}")

def baixa_aqruivos_da_amostra_rapido(sample_size, specobj_filename,download_dir):
    amostra = gerador_de_amostras(sample_size, specobj_filename)
    os.makedirs(download_dir, exist_ok=True)
    threads = []

    for i in range(sample_size):
        target_plate = amostra['PLATE'][i]
        target_mjd = amostra['MJD'][i]
        target_fiberid = amostra['FIBERID'][i]
        classe = amostra['CLASS'][i]
        subclass = amostra['SUBCLASS'][i]
        redshift = amostra['Z'][i]
        thread = threading.Thread(target=baixar_espectro, args=(target_plate, target_mjd, target_fiberid,classe, subclass, redshift, download_dir))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    print("\nProcesso de download paralelo concluído.")

In [ ]:
download_dir = 'D:\sdss_spectra' # altere para o diretório desejado

In [ ]:
for i in range(10): # baixa no total 10000 espectros, altere o range para o número desejado
    tamanho_amostra = 1000 # Defina o tamanho da amostra desejada
    catalog_file = 'specObj-dr17.fits'
    baixa_aqruivos_da_amostra_rapido(tamanho_amostra,catalog_file,download_dir)

In [ ]:
def verificar_arquivos_repitidos(diretorio):
    #avalie o começo dos arquivos
    # e verifique se eles são iguais ex spec-0266-51602-0139-STAR-F5-0.00003.fits ver apenas 0266-51602-0139
    arquivos = os.listdir(diretorio)
    arquivos_repetidos = []
    for arquivo in arquivos:
        if arquivo.endswith('.fits'):
            prefixo = arquivo.split('-')[:4]  # Pega os primeiros 4 elementos do nome do arquivo
            prefixo_str = '-'.join(prefixo)  # Junta os elementos para formar o prefixo
            if prefixo_str in arquivos_repetidos:
                print(f"Arquivo repetido encontrado: {arquivo}")
            else:
                arquivos_repetidos.append(prefixo_str)
    return arquivos_repetidos
print("Arquivos repetidos encontrados:")
repetidos = verificar_arquivos_repitidos(download_dir)

In [ ]:
import matplotlib.pyplot as plt
import glob


In [ ]:
def plot_from_directory(directory):
    # Lista todos os arquivos .fits no diretório
    
    fits_files = glob.glob(os.path.join(directory, '*.fits'))

    for fits_file in fits_files:
        # Abre o arquivo FITS
        with fits.open(fits_file) as hdul:
            # Acessa a tabela de dados
            data = hdul[1].data
            lam = 10**data['loglam']  # Converte loglam para comprimento de onda
            flux = data['flux']
            # Plota o espectro (supondo que os dados estejam na primeira coluna)
            mask = (~np.isnan(flux)) & (flux != 0)
            lam_filtrado = lam[mask]
            flux_filtrado = flux[mask]
            # Plota o espectro filtrado
# ...
            plt.figure(figsize=(10, 5))

            # --- plote normalmente ---
            plt.plot(lam_filtrado, flux_filtrado)

            # 1) grude a curva nas laterais do eixo
            plt.xlim(lam_filtrado.min(), lam_filtrado.max())  # ou plt.margins(x=0)

            # 2) reduza a margem do eixo dentro da figura
            plt.gcf().subplots_adjust(left=0.05,   # 0 = cola na borda; 0.05 dá ~5 % de margem
                                    right=0.98,  # idem na direita
                                    bottom=0.10,
                                    top=0.93)

            # 3) use tight_layout com acolchoamento mínimo
            plt.tight_layout(pad=0.1)

            # rotulagem / grade
            plt.title(f'Espectro de {os.path.basename(fits_file)}', fontsize=25)
            plt.xlabel(r'Comprimento de onda [$\mathrm{\AA}$]', fontsize=25)

            plt.ylabel(r'Fluxo $(10^{-17}\,\mathrm{erg}\,\mathrm{s}^{-1}\,\mathrm{cm}^{-2}\,\mathrm{\AA}^{-1})$', fontsize=25)

            plt.grid()

            plt.show()


In [ ]:
def print_do_lmin_e_lmax_e_num_de_pontos(directory):
    # Lista todos os arquivos .fits no diretório
    fits_files = glob.glob(os.path.join(directory, '*.fits'))

    for fits_file in fits_files:
        # Abre o arquivo FITS
        with fits.open(fits_file) as hdul:
            # Acessa a tabela de dados
            data = hdul[1].data
            lam = 10**data['loglam']  # Converte loglam para comprimento de onda
            flux = data['flux']
            # Plota o espectro (supondo que os dados estejam na primeira coluna)
            mask = (~np.isnan(flux)) & (flux != 0)
            lam_filtrado = lam[mask]
            flux_filtrado = flux[mask]
            print(f'Arquivo: {os.path.basename(fits_file)}')
            print(f'Número de pontos: {len(flux_filtrado)}')
            print(f'Comprimento de onda mínimo: {lam_filtrado.min()}')
            print(f'Comprimento de onda máximo: {lam_filtrado.max()}')
            print('---')

print_do_lmin_e_lmax_e_num_de_pontos(download_dir)

In [ ]:
# plotar dois espectros de galáxias em azul, dois de estrelas em vermelho e dois de quasares em verde, plotar um por vez

def plotar_espectros_por_classe_um_por_um(diretorio, classe):
    # Lista todos os arquivos .fits no diretório
    fits_files = glob.glob(os.path.join(diretorio, '*.fits'))
    # Filtra os arquivos pela classe desejada
    fits_files_classe = [f for f in fits_files if classe in f]
    
    for fits_file in fits_files_classe:
        # Abre o arquivo FITS
        with fits.open(fits_file) as hdul:
            # Acessa a tabela de dados
            data = hdul[1].data
            lam = 10**data['loglam']  # Converte loglam para comprimento de onda
            flux = data['flux']
            # Plota o espectro (supondo que os dados estejam na primeira coluna)
            mask = (~np.isnan(flux)) & (flux != 0)
            lam_filtrado = lam[mask]
            flux_filtrado = flux[mask]
            color = 'blue' if classe == 'GALAXY' else 'red' if classe == 'STAR' else 'green'
            plt.figure(figsize=(10, 5))
            # --- plote normalmente ---
            plt.plot(lam_filtrado, flux_filtrado, color=color)
            # 1) grude a curva nas laterais do eixo
            plt.xlim(lam_filtrado.min(), lam_filtrado.max())
            # ou plt.margins(x=0)
            # 2) reduza a margem do eixo dentro da figura
            plt.gcf().subplots_adjust(left=0.05,   # 0 = cola na borda; 0.05 dá ~5 % de margem
                                    right=0.98,  # idem na direita
                                    bottom=0.10,
                                    top=0.93)
            # 3) use tight_layout com acolchoamento mínimo
            plt.tight_layout(pad=0.1)
            # rotulagem / grade
            plt.title(f'Espectro de objeto da classe {classe}', fontsize=26)
            plt.xlabel(r'Comprimento de onda [$\mathrm{\AA}$]', fontsize=26)
            plt.ylabel(r'Fluxo $(10^{-17}\,\mathrm{erg}\,\mathrm{s}^{-1}\,\mathrm{cm}^{-2}\,\mathrm{\AA}^{-1})$', fontsize=26)
            plt.tick_params(axis='both', which='major', labelsize=26)
            plt.grid()
            plt.show()
            

In [ ]:
plotar_espectros_por_classe_um_por_um(download_dir, 'STAR')

In [ ]:
plot_from_directory(download_dir)